In [ ]:
import sys
import os
import ee
import geemap
import rasterio
import requests
from io import BytesIO
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors as colors


try:
    # Initialize the library.
#     ee.Authenticate()
    ee.Initialize()
    print('Google Earth Engine has initialized successfully!')
except ee.EEException as e:
    print('Google Earth Engine has failed to initialize!')
except:
    print("Unexpected error:", sys.exc_info()[0])
    raise
    
    


In [2]:
def add_metrics(image):
    ndvi = image.normalizedDifference(['B7', 'B4']).rename('NDVI')
    gndvi = image.normalizedDifference(['B7', 'B3']).rename('GNDVI')   
    savi = ee.Image().expression('((NIR-Red)/(NIR+Red+0.5))*1.5', {
        'NIR': image.select('B8').multiply(0.0001),
        'Red': image.select('B4').multiply(0.0001)
      }).rename('SAVI')
    return image.addBands([ndvi, gndvi, savi])

def create_gif_from_collection(image_collection,
                               clipRegion,
                               bands,
                               min_val,
                               max_val, 
                               palette: ['#640000', '#ff0000', '#ffff00', '#00c800', '#006400']):
    filmArgs = {
    "dimensions": 128,
    "region": clipRegion,
    "crs": 'EPSG:3857',
    "min": min_val, 
    "max" : max_val,
    "palette":  palette,
#     "palette":  ['#006400', '#00c800', '#ffff00', '#ff0000', '#640000'],
#     "palette": ['white', 'green'],
#     "bands": ['B4', 'B3', 'B2'],
    "bands": bands,
    "framesPerSecond": 1, 
}
    return image_collection.map(add_metrics).filterDate('2019-02-01', '2019-06-30').filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 25).getVideoThumbURL(filmArgs)

def listOfImages_printer(imagesList):
    print(len(imagesList))
    for i in range(len(imagesList)):
        if i < 10:
            print(i,"    ",imagesList[i])
        else: 
            print(i,"   ",imagesList[i])

        if (i+1) % 3 == 0:  
            print(" ")
            
def get_image_date(image): 
    return ee.Date(ee.Image(image).get('system:time_start')).format("yyyy-MM-dd").getInfo()

def show_image(Map, ImagesCollection, index, clipRegion, params):
    Images = ImagesCollection.toList(ImagesCollection.size())
    Image = ee.Image(Images.get(index)).clip(clipRegion)
    Image_metrics = add_metrics(Image).clip(clipRegion)
    print(get_image_date(Image_metrics))
    Map.addLayer(Image_metrics, params, "", True) 
    return Map

def show_image_diff(Map, ImagesCollection, index_1, index_2, clipRegion, params):
    Images = ImagesCollection.toList(ImagesCollection.size())
    Image_1 = ee.Image(Images.get(index_1)).clip(clipRegion)
    Image_1_metrics = add_metrics(Image_1)

    Image_2 = ee.Image(Images.get(index_2)).clip(clipRegion)
    Image_2_metrics = add_metrics(Image_2)
    print(get_image_date(Image_2), get_image_date(Image_1))
    Image_diff = (Image_1_metrics.subtract(Image_2_metrics)).abs().select(['NDVI', 'SAVI', 'GNDVI'])
    Map.addLayer(Image_diff, params, "", True) 
    return Map

def get_image(ImagesCollection, index, clipRegion):
    Images = ImagesCollection.toList(ImagesCollection.size())
    Image = ee.Image(Images.get(index)).clip(clipRegion)
    return ee.Image(add_metrics(Image).clip(clipRegion))
    



In [3]:
Map1 = geemap.Map(location=[52.191838, 6.741434], zoom=13)
# Map1.add_basemap('SATELLITE')
Map1


Map(center=[52.191838, 6.741434], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(…

In [192]:
fc = ee.FeatureCollection(Map1.draw_last_feature)
region = fc.geometry()
cords = region.getInfo()['coordinates']
cords

[[[6.838521, 52.214874],
  [6.834358, 52.214137],
  [6.829466, 52.211902],
  [6.824831, 52.208247],
  [6.828564, 52.206169],
  [6.835688, 52.204591],
  [6.859421, 52.20204],
  [6.855172, 52.204565],
  [6.849379, 52.208615],
  [6.845001, 52.212244],
  [6.841396, 52.213296],
  [6.838521, 52.214874]]]

In [5]:
region = ee.Geometry.Polygon([[[6.838521, 52.214874],
  [6.834358, 52.214137],
  [6.829466, 52.211902],
  [6.824831, 52.208247],
  [6.828564, 52.206169],
  [6.835688, 52.204591],
  [6.859421, 52.20204],
  [6.855172, 52.204565],
  [6.849379, 52.208615],
  [6.845001, 52.212244],
  [6.841396, 52.213296],
  [6.838521, 52.214874]]]
)

In [6]:
myCollection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterBounds(region) \
    .filterDate('2019-02-01', '2019-06-30') \
    .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 25)
listOfImages = myCollection.aggregate_array('system:index').getInfo()
print('Number of images in the collection: ', len(listOfImages))

Images = myCollection.toList(myCollection.size())



Number of images in the collection:  27


In [195]:

print(create_gif_from_collection(myCollection))

0      20190227T104019_20190227T104021_T31UGT
1      20190227T104019_20190227T104021_T31UGU
2      20190227T104019_20190227T104021_T32ULC
 
3      20190319T104019_20190319T104101_T31UGT
4      20190319T104019_20190319T104101_T31UGU
5      20190319T104019_20190319T104101_T32ULC
 
6      20190329T104029_20190329T104024_T31UGT
7      20190329T104029_20190329T104024_T31UGU
8      20190329T104029_20190329T104024_T32ULC
 
9      20190408T104029_20190408T104058_T31UGT
10     20190408T104029_20190408T104058_T31UGU
11     20190408T104029_20190408T104058_T32ULC
 
12     20190418T104029_20190418T104115_T31UGT
13     20190418T104029_20190418T104115_T32ULC
14     20190423T104031_20190423T104025_T31UGU
 
15     20190513T104031_20190513T104026_T31UGT
16     20190513T104031_20190513T104026_T31UGU
17     20190513T104031_20190513T104026_T32ULC
 
18     20190607T104029_20190607T104533_T31UGT
19     20190607T104029_20190607T104533_T31UGU
20     20190607T104029_20190607T104533_T32ULC
 
21     20190617T1040

In [8]:
visParams = {"min": 0, 
              "max": 1500, 
              "bands": ["B4", "B3", "B2"]}

ndviParams_diff = {min: 0, max: 1, "palette":  ['00cd00', 'ffff00', 'ffa500', 'cd0000', '8b0000'], "bands": ['NDVI']};



gndviParams_diff = {min: 0, max: 1, "palette":  ['00cd00', 'ffff00', 'ffa500', 'cd0000', '8b0000'], "bands": ['GNDVI']};

saviParams_diff = {min: 0, max: 1, "palette":  ['00cd00', 'ffff00', 'ffa500', 'cd0000', '8b0000'], "bands": ['SAVI']};

ndviParams = {min: -1, max: 1,     "palette":  ['#640000', '#ff0000', '#ffff00', '#00c800', '#006400'], "bands": ['NDVI']};




In [10]:
index = 6

Image_1 = ee.Image(Images.get(index)).clip(region)
Image_1_metrics = add_metrics(Image_1)
print(listOfImages[index])
Map3 = geemap.Map(location=[52.179988, 6.888958], zoom=15)
Map3.addLayer(Image_1_metrics, ndviParams, "MedianImage", True) 
Map3


20190329T104029_20190329T104024_T31UGT


Map(center=[52.179988, 6.888958], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(…

20190617T104029_20190617T104030_T31UGT 20190607T104029_20190607T104533_T31UGT


Map(center=[52.199988, 6.838958], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(…

In [13]:
Image_1 = ee.Image(Images.get(9)).clip(region)
Image_1_metrics = add_metrics(Image_1)

Image_2 = ee.Image(Images.get(6)).clip(region)
Image_2_metrics = add_metrics(Image_2)

Image_diff = (Image_1_metrics.subtract(Image_2_metrics)).abs().select(['NDVI', 'SAVI', 'GNDVI'])
print(listOfImages[3],listOfImages[6])


Map4 = geemap.Map(location=[52.199988, 6.838958], zoom=14)
Map4.addLayer(Image_diff, ndviParams_diff, "MedianImage", True) 
Map4



20190319T104019_20190319T104101_T31UGT 20190329T104029_20190329T104024_T31UGT


Map(center=[52.199988, 6.838958], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(…

In [309]:
fc = ee.FeatureCollection(Map4.draw_last_feature)
cords = fc.geometry().getInfo()['coordinates']
cords

In [15]:
region_usselo_2_234_135_2019_glyph_corn = ee.Geometry.Polygon([[[6.844, 52.209074],
  [6.843228, 52.20818],
  [6.845138, 52.20724],
  [6.846114, 52.208002],
  [6.844387, 52.209192],
  [6.844, 52.209074]]])

region_usselo_1_293_084_2019_glyph = ee.Geometry.Polygon([[[6.848914, 52.206803],
  [6.849451, 52.20604],
  [6.849697, 52.205863],
  [6.849944, 52.205836],
  [6.849955, 52.205646],
  [6.850245, 52.205613],
  [6.850234, 52.205323],
  [6.8515, 52.205258],
  [6.851468, 52.205777],
  [6.851189, 52.206441],
  [6.851189, 52.206901],
  [6.85106, 52.20727],
  [6.848914, 52.206803]]])

region_usselo_Usseleresweg_293_084_2019_glyph = ee.Geometry.Polygon([[[6.833486, 52.207986],
  [6.829935, 52.20721],
  [6.833443, 52.207099],
  [6.833347, 52.206882],
  [6.8331, 52.206408],
  [6.835364, 52.206329],
  [6.835503, 52.207158],
  [6.833712, 52.207296],
  [6.833486, 52.207986]]])


region_usselo_Usselerscholweg_293_084_2019_glyph = ee.Geometry.Polygon([[[6.850706, 52.207381],
  [6.849032, 52.206698],
  [6.850234, 52.205475],
  [6.851403, 52.205455],
  [6.850706, 52.207381]]]) #06-17 to 06-22 too!


region_usselo_Ruilverkavelingsweg_293_084_2019_glyph = ee.Geometry.Polygon([[[6.840021, 52.206288],
  [6.836459, 52.207379],
  [6.836566, 52.206959],
  [6.837961, 52.206709],
  [6.838025, 52.205525],
  [6.839957, 52.205486],
  [6.840021, 52.206288]]])

region_usselo_Haaksbergerstraat_293_084_2019_glyph = ee.Geometry.Polygon([[[6.837188, 52.204605],
  [6.849098, 52.203145],
  [6.849269, 52.204907],
  [6.84796, 52.20496],
  [6.847789, 52.203434],
  [6.844763, 52.203763],
  [6.844935, 52.204868],
  [6.839399, 52.205039],
  [6.837188, 52.204605]]])

region_usselo_Ruilverkavelingsweg2_184_084_2019_glyph = ee.Geometry.Polygon([[[6.843497, 52.210995],
  [6.842961, 52.210233],
  [6.845171, 52.209233],
  [6.845815, 52.209996],
  [6.843497, 52.210995]]]
) 

# Most site, also over to 135!!!




In [16]:
collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterBounds(region_usselo_Ruilverkavelingsweg2_184_084_2019_glyph) \
    .filterDate('2019-03-01', '2019-06-30') \
    .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 25)
listOfImages = collection.aggregate_array('system:index').getInfo()
listOfImages_printer(listOfImages)


24
0      20190319T104019_20190319T104101_T31UGT
1      20190319T104019_20190319T104101_T31UGU
2      20190319T104019_20190319T104101_T32ULC
 
3      20190329T104029_20190329T104024_T31UGT
4      20190329T104029_20190329T104024_T31UGU
5      20190329T104029_20190329T104024_T32ULC
 
6      20190408T104029_20190408T104058_T31UGT
7      20190408T104029_20190408T104058_T31UGU
8      20190408T104029_20190408T104058_T32ULC
 
9      20190418T104029_20190418T104115_T31UGT
10     20190418T104029_20190418T104115_T32ULC
11     20190423T104031_20190423T104025_T31UGU
 
12     20190513T104031_20190513T104026_T31UGT
13     20190513T104031_20190513T104026_T31UGU
14     20190513T104031_20190513T104026_T32ULC
 
15     20190607T104029_20190607T104533_T31UGT
16     20190607T104029_20190607T104533_T31UGU
17     20190607T104029_20190607T104533_T32ULC
 
18     20190617T104029_20190617T104030_T31UGT
19     20190617T104029_20190617T104030_T31UGU
20     20190617T104029_20190617T104030_T32ULC
 
21     20190622T1

In [22]:
Map = geemap.Map(location=[52.20496, 6.84796], zoom=15)
show_image(Map, collection, 15, region_usselo_Ruilverkavelingsweg2_184_084_2019_glyph, ndviParams)
# show_image_diff(Map, collection, 18, 21, region_usselo_234_135_2019_glyph, ndviParams_diff)

2019-06-07


Map(center=[52.20496, 6.84796], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(ch…

In [ ]:
image = get_image(collection, 15, region_usselo_Ruilverkavelingsweg2_184_084_2019_glyph)
field = "Ruilverkavelingsweg2/"
filename = "Usselo-Enschede/" + field + get_image_date(image) + ".tif"
geemap.ee_export_image(
    image,
    filename=filename, 
    scale=30, 
    region=region_usselo_Ruilverkavelingsweg2_184_084_2019_glyph, 
    file_per_band=False
)

##### 